In [1]:
import torch
import numpy as np
from datasets import load_dataset
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
batch_size = 256

# Creating vocabulory

In [3]:
n = 3
embedding_dims = 10 # how many dimensional vector should represent each word in the vocabulory

In [4]:
ds = load_dataset("MuskumPillerum/General-Knowledge")
df = ds['train'][:10]

In [5]:
def tokenize(sentence):
    data = sentence
    split_terms = [',', '.', '!', '?', '(', ')', '&', '$', '+', '-', '/', '*', ';', ':']
    for split_term in split_terms:
        if split_term in sentence:
            data = data.replace(split_term, f' {split_term} ')
    data = data.split()
    return data

tokenize('what is your !! ! name ?    ')

['what', 'is', 'your', '!', '!', '!', 'name', '?']

In [6]:
vocab_list = set()
X = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'Question: {x} Answer: {y}'
    data = data.lower().replace('\\n', '')
    vocab_list.update(tokenize(data))
    X.append(data)

In [7]:
vocab_list.add('<UNK>')

In [8]:
vocab = {v:i for v,i in zip(vocab_list, range(0, len(vocab_list)+1))}
len(vocab)

141

In [9]:
vocab

{'neural': 0,
 'networks': 1,
 'type': 2,
 'main': 3,
 'typically': 4,
 'by': 5,
 'system': 6,
 'where': 7,
 'brain': 8,
 'inspired': 9,
 'over': 10,
 'like': 11,
 'vision': 12,
 'learn': 13,
 'algorithm': 14,
 'task': 15,
 'way': 16,
 'focuses': 17,
 'learns': 18,
 'examples': 19,
 'around': 20,
 'translation': 21,
 'aims': 22,
 'an': 23,
 'while': 24,
 ',': 25,
 'complex': 26,
 'without': 27,
 'layers': 28,
 'interconnected': 29,
 '?': 30,
 'nodes': 31,
 ':': 32,
 'labeled': 33,
 'various': 34,
 'uses': 35,
 'given': 36,
 '.': 37,
 'simulate': 38,
 'development': 39,
 '-': 40,
 'artificial': 41,
 'two': 42,
 'such': 43,
 'computers': 44,
 'this': 45,
 '<UNK>': 46,
 'human': 47,
 'in': 48,
 'or': 49,
 'that': 50,
 'relationships': 51,
 'make': 52,
 'patterns': 53,
 'penalty': 54,
 'learning': 55,
 'on': 56,
 'language': 57,
 'generate': 58,
 'works': 59,
 'form': 60,
 'transmit': 61,
 'making': 62,
 'consists': 63,
 'speech': 64,
 'based': 65,
 'allows': 66,
 'to': 67,
 'perception': 

In [10]:
embedding_matrix = torch.randn(len(vocab),embedding_dims)
embedding_matrix.shape

torch.Size([141, 10])

In [11]:
embedding_matrix[[1,2,3]]

tensor([[ 0.8063, -0.6507,  0.3987,  1.6612,  0.7751, -0.0549,  1.3940,  1.7015,
          0.3626, -0.5787],
        [-0.5516,  0.9090, -1.4153, -0.1322, -0.0741, -2.0019, -0.3743,  0.4923,
         -0.2098,  1.6077],
        [-1.7951,  0.0110, -1.4094, -0.4328,  0.2410,  0.3412, -0.7195, -0.6139,
         -0.1315, -0.0925]])

In [12]:
def get_word_embedding(word, vocab=vocab, embedding_matrix=embedding_matrix):
    if word not in vocab:
        word = '<UNK>'
    embedding = embedding_matrix[vocab[word]]
    return embedding

get_word_embedding('as')

tensor([-0.1914, -2.0334,  1.6065,  0.2783,  0.8104,  1.7177,  0.6526, -0.9009,
         1.5135, -0.1461])

# Creating the model

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self, n=n, num_hidden_layer=1024, vocab = vocab, dim_embedding=10):
        super(NeuralNetwork, self).__init__()
        vocab_len = len(vocab)
        self.n = n
        self.dim_embedding = dim_embedding
        self.embedding = nn.Embedding(vocab_len, dim_embedding).to(device)

        self.hidden_layer = nn.Linear((n-1)*dim_embedding, num_hidden_layer)
        self.relu = nn.ReLU()
        self.output = nn.Linear(num_hidden_layer, vocab_len)

    def forward(self,x):
        # x will be the indices of embedding representing the input words
        x_embeddings = self.embedding(x).view(-1,(self.n-1)*self.dim_embedding).to(device)
        out = self.hidden_layer(x_embeddings)
        out = self.relu(out)
        out = self.output(out)
        print(out.shape)
        return out
    

    def generate(self, x):
        logits = self.forward(x)
        probs = torch.softmax(logits, dim=1)
        prob, predicted_token_index = torch.max(probs, dim=1)

        return predicted_token_index

model = NeuralNetwork(n=3).to(device)

# Creating dataset

In [14]:
dataset = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'Question: {x} Answer: {y}'.lower()
    tokenized_data = tokenize(data)
    for i in range(len(tokenized_data)-n):
        # print(i)
        data_i = tokenized_data[i:i+n]
        dataset.append([vocab[i] if i in vocab else vocab['<UNK>'] for i in data_i])
    # print()

dataset_np = np.array(dataset)
dataset_np.shape

(419, 3)

In [15]:
class QuestionAnswerDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.x = dataset[:,[i for i in range(n-1)]]
        self.y = dataset[:,-1]
        self.m, self.n = self.x.shape
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.m
    
dataset = QuestionAnswerDataset(dataset=dataset_np)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Training

In [16]:
num_epochs = 100
lr = 0.01
print(device)

cuda


In [17]:
model = model.to(device)
model = torch.compile(model)

In [18]:
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=lr)

In [19]:
for epoch in range(num_epochs):
    for i, (input, label) in enumerate(dataloader):
        x = input.to(device)
        y = label.to(device)
        y_pred = model(x)
        loss = lossCategory(y_pred, y)
        if epoch%10==0 and i==0: 
            # print(epoch)
            print(loss.item())
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()


/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:150: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


torch.Size([256, 141])
4.981202125549316
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
4.739145755767822
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
torch.Size([163, 141])
torch.Size([256, 141])
4.494852542877197
tor

# Inference

In [20]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [21]:
vocab['is']

98

In [42]:
text = 'What is a ?'
tokenized_text = tokenize(text.lower())
embedding_indices = torch.tensor(np.array([vocab[word] for word in tokenized_text])).to(device)
embedding_indices

tensor([ 69,  98, 101,  30], device='cuda:0')

In [43]:
embedding_indices

tensor([ 69,  98, 101,  30], device='cuda:0')

In [44]:
model.generate(embedding_indices)

torch.Size([2, 141])


tensor([101,  86], device='cuda:0')

In [45]:
vocab_rev = {v:k for k,v in vocab.items()}

In [47]:
vocab_rev[86]

'answer'

In [27]:
model.embedding

Embedding(141, 10)

In [28]:
# TO DO
# normalisation, standardise, proper init, weight decay, hyperparameter tunign, diff optimiser test, inference, earlyy stopping
# add direct connections, see if it is useful
# plot unigram, bi, tri, etc perplexity score, word error rate, etc